## Introduction

Knowledge bases in enterprises are very common in the industry today and can have extensive number of documents in different categories. Retrieving relevant content based on a user query is a challenging task.  Given a query we were able to retrieve information accurately at the document level using methods such as Page Rank developed and made highly accurate especially by Google,  after this point the user has to delve into the document and search for the relevant information.  With recent advances in Foundation Models such as the one developed by Open AI the challenge is alleviated by using “Semantic Search” methods by using encoding information such as “Embeddings” to find the relevant information and then to summarize the content to present to the user in a concise and succinct manner.  

This notebook will introduce the Use Case and will walk you through leveraging Azure Cognitive Search to extract relevant documents and leveraging the power of GPT-3 to address relevant part of the document, and provide a summary based on the prompt (instruction given to the model). It aims to demonstrate how to use Azure OpenAI’s GPT-3 capabilities to adapt to your summarization case, and how to set up and evaluate summarization results. The method is customizable to your summarization use case and can be applied to many different datasets. 

## Use Case

This use case consists of three sections:
- Document search
- Document Zone search
- Text summarization

Document Search is the process of extracting relevant document based on the query from a corpus of documents.
Document Zone search is the process of finding the relevant part of the document extracted from document search.
Text summarization is the process of creating summaries from large volumes of data while maintaining significant informational elements and content value. 
This use case can be useful in helping subject matter experts in finding relevant information from large document corpus.
Example: In the drug discovery process, scientists in pharmaceutical industry read a corpus of documents to find specific information related to concepts, experiment results etc. This use case enables them to ask questions from the document corpus and the solution will come back with the succinct answer. Consequently, expediting the drug discovery process.
 
Benefits of the solution:
1. Shortens reading time
2. Improves the effectiveness of searching for information
3. Removes bias from human summarization techniques
4. Increases bandwidth for humans to focus on more in-depth analysis 


The need for document summarization be applied to any subject matter (legal, financial, journalist, medical, academic, etc) that requires long document summarization. The subject matter that this notebook is focusing on is journalistic - we will walk through news articles. If the topic gets more domain specific, fine-tuning of the GPT3-model would work better rather than just using the few-shot or zero-shot example methods.  


## CNN daily mail dataset
For this walkthrough, we will be using the CNN/Daily Mail dataset. This is a common dataset used for text summarization and question answering tasks. Human generated abstractive summary bullets were generated from news stories in CNN and Daily Mail websites.


## Data Description
The relevant schema for our work today consists of:

- id: a string containing the heximal formatted SHA1 hash of the URL where the story was retrieved from
- article: a string containing the body of the news article
- highlights: a string containing the highlight of the article as written by the article author


## Import python modules

In [1]:
import json
import requests
from pprint import pprint

In [5]:
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
import openai
import re
import requests
import sys
import os
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
#from transformers import GPT2TokenizerFast
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

model=os.getenv("DEPLOYMENT_NAME")
assert model, "ERROR: Deployment_Name is missing"

In [2]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "news-ds"
#skillset_name = "news-skills"
index_name = "cnn-news-index"
indexer_name = "cnn-news-indexer"

In [12]:
endpoint = os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT")
headers = {'Content-Type': 'application/json',
           'api-key': os.getenv("AZURE_COGNITIVE_SEARCH_KEY")}
params = {
    'api-version': os.getenv('AZURE_COGNITIVE_SEARCH_API_VERSION')
}

In [4]:

# Create a data source
# This data source points to your Azure Storage account.
# You should already have a blob container that contains the sample data
datasourceConnectionString = os.getenv("AZURE_BLOB_STORAGE_CONNECTION_STRING")
datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": datasourceConnectionString
    },
    "container": {
        "name": "news-data"
    }
}
r = requests.put(endpoint + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)

201


In [7]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
index_payload = {
    "name": index_name,
    "fields": [
        {
            "name": "id",
            "type": "Edm.String",
            "key": "true",
            "searchable": "false",
            "filterable": "false",
            "facetable": "false",
            "sortable": "false"
        },
        {
            "name": "article",
            "type": "Edm.String",
            "sortable": "false",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false",
            "analyzer": "standard.lucene"
        },
        {
            "name": "highlights",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false",
            "analyzer": "standard.lucene"
        }
    ]
}

r = requests.put(endpoint + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)

201


In [8]:
indexer_payload = {
  "name" : indexer_name,
  "dataSourceName" : datasource_name,
  "targetIndexName" : index_name,
  "parameters" : { 
      "configuration" : { 
          "parsingMode" : "delimitedText", "delimitedTextHeaders" : "id,article,highlights" 
      }
  }
}



r = requests.put(endpoint + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)

201


In [7]:
# read the CNN dailymail dataset in pandas dataframe
df = pd.read_csv('data/cnn_dailymail_data.csv') #path to CNN daily mail dataset
df.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


## Section 1: Leveraging Cognitivie search to extract relevant article based on the query 

## Creating Cognitive Seach Index using CNN Dailymail dataset
<img src="../Demo/images/AzureCogSearchIndex.png" alt="Alternative text" />

In [16]:
# Create an SDK client
search_client = SearchClient(endpoint=os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT"),
                      index_name=os.getenv('AZURE_COGNITIVE_SEARCH_INDEX_NAME'),
                      api_version=os.getenv('AZURE_COGNITIVE_SEARCH_API_VERSION'),
                      credential=AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_KEY")))

In [39]:
search_term = "Stock market news"
count = 2
results = search_client.search(search_text=search_term, top=count, include_total_count=True)
for result in results:
    print(result)
    print("\n\n")

{'id': '9f467b20eab77fe837f043d1577f75bd325c5d37', 'highlights': 'Shares have performed nearly twice as well under Tories, new figures show .\r\nStock market returns averaged 16 per cent per year under Conservatives .\r\nReturns hovered around nine per cent under Labour and current Coalition .', 'article': "Stock market returns have averaged 16 per cent a year under David Cameron's Conservatives party, compared to just under nine per cent under Labour . Shares have performed nearly twice as well under Conservative governments than under Labour over the last 45 years, according to figures published today. Stock market returns have averaged 16 per cent a year under the Tories compared with just under 9 per cent under Labour and just over 9 per cent under the current Coalition. The figures, in an analysis for the Daily Mail by investment company Hargreaves Lansdown, are a further boost for David Cameron and George Osborne ahead of the general election on May 7. Ed Miliband is struggling t

In [18]:
#Extracting relevant article based on the query. eg: Clinton Democratic Nomination
results = search_client.search(search_text="Clinton Democratic nomination", include_total_count=True)
document = next(results)['article']

In [19]:
document

'Apple founder Steve Jobs\' widow Laurene has told of her admiration for Democratic White House front-runner Hillary Clinton. Ms Jobs, 51, called former First Lady Hillary a \'revolutionary\' woman, and added that it\'s not just because she\'s a woman - but \'the type of woman she is\'. Speaking to Time 100, Ms Jobs said: \'Hillary Clinton is not familiar. She is revolutionary. Not radical, but revolutionary: The distinction is crucial. She is one of America’s greatest modern creations. Laurene Jobs, pictured, widow of Apple\'s Steve, has strongly backed Hillary Clinton for president . Laurene Jobs said that Hillary Clinton, right, has \'judgment and wisdom\' based on her public service . \'Her decades in our public life must not blind us to the fact that she represents new realities and possibilities. Indeed, those same decades have conferred upon her what newness usually lacks: judgment, and even wisdom. \'It matters, of course, that Hillary is a woman. But what matters more is what 

In [20]:
#length of article extracted from Azure Cognitive search
len(document) 

6675

## Section 2: Document Zone Search
Document Zone: Azure OpenAI Embedding API
Now that we narrowed on a single document from our knowledge base using Azure Cognitive Search- we can dive deeper into the single document to refine our initial query to a more specific section or "zone" of the article.

To do this, we will utilize the Azure Open AI Embeddings API.

## Embeddings Overview
An embedding is a special format of data representation that can be easily utilized by machine learning models and algorithms. The embedding is an information dense representation of the semantic meaning of a piece of text. Each embedding is a vector of floating-point numbers, such that the distance between two embeddings in the vector space is correlated with semantic similarity between two inputs in the original format. For example, if two texts are similar, then their vector representations should also be similar.

Different Azure OpenAI embedding models are specifically created to be good at a particular task. Similarity embeddings are good at capturing semantic similarity between two or more pieces of text. Text search embeddings help measure long documents are relevant to a short query. Code search embeddings are useful for embedding code snippets and embedding nature language search queries.

Embeddings make it easier to do machine learning on large inputs representing words by capturing the semantic similarities in a vector space. Therefore, we can use embeddings to if two text chunks are semantically related or similar, and inherently provide a score to assess similarity.

## Cosine Similarity
A previously used approach to match similar documents was based on counting maximum number of common words between documents. This is flawed since as the document size increases, the overlap of common words increases even if the topics differ. Therefore cosine similarity is a better approach.

Mathematically, cosine similarity measures the cosine of the angle between two vectors projected in a multi-dimensional space. This is beneficial because if two documents are far apart by Euclidean distance because of size, they could still have a smaller angle between them and therefore higher cosine similarity.

The Azure OpenAI embeddings rely on cosine similarity to compute similarity between documents and a query.

## Setting up Azure OpenAI service and using deployed models

In [21]:
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

TEXT_SEARCH_DOC_EMBEDDING_ENGINE = os.getenv('DOCUMENT_MODEL_NAME') # Model deployment name - mentioned in the above screenshot 
TEXT_SEARCH_QUERY_EMBEDDING_ENGINE = os.getenv('QUERY_MODEL_NAME') # Model deployment name - mentioned in the above screenshot
TEXT_DAVINCI = os.getenv('DEPLOYMENT_NAME') # Model deployment name - mentioned in the above screenshot

In [22]:
#Defining helper functions
#Splits text after sentences ending in a period. Combines n sentences per chunk.
def splitter(n, s):
    pieces = s.split(". ")
    list_out = [" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n)]
    return list_out

# Perform light data cleaning (removing redudant whitespace and cleaning up punctuation)
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()    
    return s

In [23]:
document_chunks = splitter(10, normalize_text(document)) #splitting extracted document into chunks of 10 sentences
document_chunks

["Apple founder Steve Jobs' widow Laurene has told of her admiration for Democratic White House front-runner Hillary Clinton Ms Jobs, 51, called former First Lady Hillary a 'revolutionary' woman, and added that it's not just because she's a woman - but 'the type of woman she is' Speaking to Time 100, Ms Jobs said: 'Hillary Clinton is not familiar She is revolutionary Not radical, but revolutionary: The distinction is crucial She is one of America’s greatest modern creations Laurene Jobs, pictured, widow of Apple's Steve, has strongly backed Hillary Clinton for president  Laurene Jobs said that Hillary Clinton, right, has 'judgment and wisdom' based on her public service  'Her decades in our public life must not blind us to the fact that she represents new realities and possibilities Indeed, those same decades have conferred upon her what newness usually lacks: judgment, and even wisdom",
 "'It matters, of course, that Hillary is a woman But what matters more is what kind of woman she i

In [25]:
embed_df = pd.DataFrame(document_chunks, columns = ["chunks"]) #datframe with document chunks


In [26]:
#Create an embedding vector for each chunk that will capture the semantic meaning and overall topic of that chunk
embed_df['davinci_search'] = embed_df["chunks"].apply(lambda x : get_embedding(x, engine = TEXT_SEARCH_DOC_EMBEDDING_ENGINE))

In [27]:
embed_df #dataframe with document chunks and their embeddings created using Curie embeddings model 

,chunks,davinci_search
0,Apple founder Steve Jobs' widow Laurene has to...,"[-0.00974290445446968, 0.0023548523895442486, ..."
1,"'It matters, of course, that Hillary is a woma...","[0.012613990344107151, 0.00011710495164152235,..."
2,Bird himself is a frequent participant in Iowa...,"[0.0016598827205598354, 0.002051843563094735, ..."
3,Price was executive director of the Iowa Democ...,"[0.008494426496326923, -0.005251964554190636, ..."
4,And planting party insiders in place of typica...,"[0.014864214695990086, -0.003792150178924203, ..."
5,'I was driving the Vice President when he was ...,"[0.004335296340286732, -0.015600714832544327, ..."


In [28]:
# search through the document for a text segment most similar to the query
# display top two most similar chunks based on cosine similarity
def search_docs(df, user_query, top_n=3):
    embedding = get_embedding(
        user_query,
        engine=TEXT_SEARCH_QUERY_EMBEDDING_ENGINE
    )
    df["similarities"] = df.davinci_search.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .reset_index(drop=True)
        .head(top_n)
    )
    return res

In [29]:
document_specific_query = "trouble so far in clinton campaign" 
res = search_docs(embed_df, document_specific_query, top_n=2) #finding top 2 results based on similarity 
res

,chunks,davinci_search,similarities
0,Price was executive director of the Iowa Democ...,"[0.008494426496326923, -0.005251964554190636, ...",0.293145
1,"'It matters, of course, that Hillary is a woma...","[0.012613990344107151, 0.00011710495164152235,...",0.288064


## Section 3: Summarizer

This section will cover the end-to-end flow of using the GPT-3 models for summarization tasks. 
The model used by the Azure OpenAI service is a generative completion call which uses natural language instructions to identify the task being asked and skill required – aka Prompt Engineering. Using this approach, the first part of the prompt includes natural language instructions and/or examples of the specific task desired. The model then completes the task by predicting the most probable next text. This technique is known as "in-context" learning. 

There are three main approaches for in-context learning: Zero-shot, Few-shot and Fine tuning. These approaches vary based on the amount of task-specific data that is given to the model: 

**Zero-shot**: In this case, no examples are provided to the model and only the task request is provided. 

**Few-shot**: In this case, a user includes several examples in the call prompt that demonstrate the expected answer format and content. 

**Fine-Tuning**: Fine Tuning lets you tailor models to your personal datasets. This customization step will let you get more out of the service by providing: 
-	With lots of data (at least 500 and above) traditional optimization techniques are used with Back Propagation to re-adjust the weights of the model – this enables higher quality results than mere zero-shot or few-shot. 
-	A customized model improves the few-shot learning approach by training the model weights on your specific prompts and structure. This lets you achieve better results on a wider number of tasks without needing to provide examples in the prompt. The result is less text sent and fewer tokens 


In [34]:
'''Designing a prompt that will show and tell GPT-3 how to proceed. 
+ Providing an instruction to summarize the text about the general topic (prefix)
+ Providing quality data for the chunks to summarize and specifically mentioning they are the text provided (context + context primer)
+ Providing a space for GPT-3 to fill in the summary to follow the format (suffix)
'''

# result_1 corresponding to the top chunk from Section 2. result_2 corresponding to the second to top chunk from section 2. 
# change index for desired chunk
result_1 = res.chunks[0]
result_2 = res.chunks[1]
prompt_i = 'Summarize the content about the Clinton campaign given the text provided.\n\nText:\n'+" ".join([normalize_text(result_1)])+ '\n\nText:\n'+ " ".join([normalize_text(result_2)])+'\n\nSummary:\n'
print(prompt_i)

Summarize the content about the Clinton campaign given the text provided.

Text:
Price was executive director of the Iowa Democratic Party until a month ago Clinton's team tapped him last week to be its political director in Iowa He did not respond to a request for comment Bird is a government and community relations coordinator at Genesis Health System in Davenport, Iowa, according to his LinkedIn profile A coworker at Genesis said Wednesday that Bird is 'basically a lobbyist in training That's what he wants to do.' Bird disagreed, saying his role was 'more public relations.' He's also an outspoken progressive whose Facebook wall shows he ordered a 'Hillary For President' bumper sticker 22 months ago 'Is it 2016 yet?' he wrote in May 2013 Clinton's nascent campaign has carefully coordinated her image as a spontaneous, handshaking populist in her first days as a candidate, posing with Pennsylvanians at a gas station and venturing into an Ohio Chipotle restaurant for lunch When no one r

In [30]:

# Using a low temperature to limit the creativity in the response. 
response = openai.Completion.create(
        engine= TEXT_DAVINCI,
        prompt = prompt_i,
        temperature = 0.0,
        max_tokens = 500,
        top_p = 1.0,
        frequency_penalty=0.5,
        presence_penalty = 0.5,
        best_of = 1
    )

print(response.choices[0].text)

Hillary Clinton's campaign has been carefully coordinating her image as a spontaneous, handshaking populist in her first days as a candidate. During her trip to Iowa, she was criticized for staging an event at a coffee shop with three young Iowans who were driven there by the Iowa campaign's political director. The three got the lion's share of Mrs Clinton's time and participated in what was described as a 'roundtable'. This scripting of supposedly off-the-cuff appearances is common in presidential politics but could hurt Clinton especially hard since her road-trip journey to America's broad midwest is designed to counter her image as cold, calculating and politically venomous.


In [31]:
#testing some parameters with a differnt temperature
response = openai.Completion.create(
        engine= TEXT_DAVINCI,
        prompt = prompt_i,
        temperature = 0.2,
        max_tokens = 500,
        top_p = 1.0,
        frequency_penalty=0.5,
        presence_penalty = 0.5,
        best_of = 1
    )

print(response.choices[0].text)

Hillary Clinton's campaign has been carefully coordinating her image as a spontaneous, handshaking populist in her first days as a candidate. During her trip to Iowa, she was criticized for staging an event at a coffee shop with three young people who were driven there by the Iowa campaign's political director. The three got the lion's share of Mrs Clinton's time and participated in what was described as a 'roundtable'. Her team is attempting to counter her image as cold, calculating and politically venomous.


In [32]:
#testing some parameters 
response = openai.Completion.create(
        engine= TEXT_DAVINCI,
        prompt = prompt_i,
        temperature = 0.7,
        max_tokens = 500,
        top_p = 1.0,
        frequency_penalty=0.5,
        presence_penalty = 0.5,
        best_of = 1
    )

print(response.choices[0].text)

Hillary Clinton began her campaign for the Democratic nomination with a visit to Iowa, where she was met with criticism for her 'staged' appearance at a coffee shop in LeClaire. Austin Bird, one of the people pictured sitting with Clinton, revealed that the event had been orchestrated from beginning to end by her Iowa campaign's political director, Troy Price. Bird and two other young people were driven to the coffee house by Price, who vetted them for around half an hour before they met Clinton. The three participants received most of Clinton's time and took part in what was described as a 'roundtable'.
